In [1]:
import pandas as pd
import math

1. Функция, которая приводит текст к нижнему регистру и заменяет ё на е

In [17]:
def norm(s): 
    return str(s).lower().replace("ё", "е")

2. Сборка словаря "слово-стоимость" из csv файла

In [18]:
dfc = pd.read_csv("word_forms_with_cost.csv")
dfc["word_form"] = dfc["word_form"].astype(str).map(norm)
dfc["cost"] = pd.to_numeric(dfc["cost"], errors="coerce")
word2cost = dict(zip(dfc["word_form"], dfc["cost"]))

3. Алфавиты и базовые множества

In [19]:
ru = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
en = "abcdefghijklmnopqrstuvwxyz"
letters_ru = set(ru + ru.upper())
letters_en = set(en + en.upper())
digits = set("0123456789")
punct_need_space = set(",?!;:") #пунктуация, после которой нужен пробел
whitelist = set(map(norm, "я а в и то ты так но да не ни о от до за у к с по на во ко со б бы об из без под над при для как же ли же ну уж".split()))
letters = letters_ru | letters_en

4. Гиперпараметры штрафов для стоимостей

In [20]:
new_word_penalty = 80.0
penalty_per_char = 1.0
split_penalty = 1.0
short_penalty = 10.0

5. Функции определения типа токена

In [21]:
def is_word(tok):
    return tok and all(ch in letters for ch in tok)

def is_digit(tok):
    return tok and all(ch in digits for ch in tok)

6. Функция определения стоимости токена, включая штрафы: 
+ штраф за разделение на короткие слова - short_penalty (кроме фикс списка слов - whitelist)
+ штраф за слово, несуществующее в словаре - new_word_penalty
+ штраф за длину - penalty_per_char

In [22]:
def token_cost(tok: str) -> float:
    t = norm(tok)
    if t in word2cost:
        c = float(word2cost[t])
        if len(t) <= 2 and t not in whitelist:
            c += short_penalty
        return c
    if t.isdigit():
        return 2.0 + 0.1 * len(t)
    return new_word_penalty + penalty_per_char * len(t)

7. Вспомогательные функции для классификации "-" дефисом или тире

+ суффиксы, приставки, предлоги через дефис
+ функция alphnum - является ли буквой или цифрой
+ функция alphnum_block_left - блок слева от "-"
+ функция alphnum_block_right - блок справа от "-"

In [23]:
suffixes = {"то", "либо", "нибудь", "таки", "ка"}
prefixes   = {"кое", "по", "из", "изо", "под", "над", "меж", "супер", "анти",
                    "ультра", "микро", "нано", "кило", "мега", "квази", "псевдо",
                    "вице", "эко", "био", "термо", "авто", "сверх", "суб", "пан"}
special_pairs = {("из", "за"), ("из", "под"), ("по", "над")}
hyphen_cache = {}

def alphnum(ch: str) -> bool:
    return (ch in letters) or (ch in digits)

def alphnum_block_left(s: str, idx_minus: int) -> str:
    j = idx_minus - 1
    while j >= 0 and alphnum(s[j]):
        j -= 1
    return s[j+1:idx_minus]

def alphnum_block_right(s: str, idx_minus: int) -> str:
    j = idx_minus + 1
    n = len(s)
    while j < n and alphnum(s[j]):
        j += 1
    return s[idx_minus+1:j]

8. Функция классификация "-" дефисом или тире - classify_minus

In [24]:
def classify_minus(s: str, boundary_i: int) -> str:

    #boundary_i — позиция - граница между s[i-1] и s[i].
    #Возвращает 'hyphen' (дефис, без пробелов) или 'dash' (тире, с пробелами).
    #Логика: по умолчанию 'dash', 'hyphen' только определенных условиях

    # Ключ = (позиция, символы вокруг)
    key = (boundary_i, s[max(0, boundary_i-8):boundary_i+8])
    if key in hyphen_cache:
        return hyphen_cache[key]

    # Находим индекс "-"
    if s[boundary_i-1] == "-":
        idx_minus = boundary_i-1
    else:
        idx_minus = boundary_i
    
    # Определение левого и правого символа
    if idx_minus - 1 >= 0:
        left_char = s[idx_minus-1]
    else:
        left_char = None

    if idx_minus + 1 < len(s):
        right_char = s[idx_minus+1]
    else:
        right_char = None

    # Если справа или слева ничего нет ИЛИ справа или слева не буква/цифра, то ТИРЕ
    if left_char is None or right_char is None or (not alphnum(left_char)) or (not alphnum(right_char)):
        hyphen_cache[key] = "dash"
        return "dash"

    # Выделяем блоки слева и справа
    left = alphnum_block_left(s, idx_minus)
    right = alphnum_block_right(s, idx_minus)
    l = norm(left)
    r = norm(right)

    # Если есть стоимость слова l-r меньше 25, то ДЕФИС
    if left and right:
        cand = f"{l}-{r}"
        cost = word2cost.get(cand)
        if cost is not None and cost < 25:
            hyphen_cache[key] = "hyphen"
            return "hyphen"

    # Если l и r образуют предлог, то ДЕФИС
    if (l, r) in special_pairs:
        hyphen_cache[key] = "hyphen"
        return "hyphen"

    # Если приставки / суффиксы, то ДЕФИС
    if r in suffixes and left.isalpha():
        hyphen_cache[key] = "hyphen"
        return "hyphen"
    if l in prefixes and right.isalpha():
        hyphen_cache[key] = "hyphen"
        return "hyphen"

    # Если буква-цифра или цифра-буква (модели, индексы), то ДЕФИС
    if (left.isalpha() and right.isdigit()) or (left.isdigit() and right.isalpha()):
        hyphen_cache[key] = "hyphen"
        return "hyphen"

    # По умолчанию ТИРЕ
    hyphen_cache[key] = "dash"
    return "dash"

9. Функция восстановления пробелов - space_positions

In [25]:
def space_positions(text: str, max_tok_len: int = 25):
    # Возращает список индексов, где нужно поставить пробел
    
    # Два блока:
    # 1. Правила по символам, собираем два множества (yes_space - пробел обязателен, no_space - пробел запрещен)
    # 2.1. Подготовка к динамическому программированию (разделение на английские/русские буквы /цифры /другое)
    # 2.2. Динамическое программирование на русских буквах (минимизируем суммарную стоимость токенов + штрафы за разрезы)
    # Расставляем пробелы между токенами
    # Итоговое расположение пробелов = (Границы между токенами объединенные с yes_space, удаляя no_space позиции) + сортировка

    s = str(text)
    n = len(s)

    yes_space = set()
    no_space = set()

    def is_ru(ch): return ch in letters_ru
    def is_en(ch): return ch in letters_en
    def is_letter(ch): return ch in letters
    def is_dig(ch): return ch in digits

    # 1. Правила по символам
    for i in range(1, n):
        a = s[i-1]
        b = s[i]

        # Если цифра/буква ИЛИ буква/цифры, то ПРОБЕЛ
        if (is_dig(a) and is_letter(b)) or (is_letter(a) and is_dig(b)):
            yes_space.add(i)

        # Если смена алфавита, то ПРОБЕЛ
        if (is_ru(a) and is_en(b)) or (is_en(a) and is_ru(b)):
            yes_space.add(i)

        # Если знак пунктуации, то после ПРОБЕЛ
        if a in punct_need_space:
            yes_space.add(i)

        # Если одна точка (не многоточие), то после ПРОБЕЛ
        if a == "." and b != ".":
            yes_space.add(i)

        # Если после многоточия буква/цифра, то после ПРОБЕЛ
        if i >= 3 and s[i-3:i] == "..." and alphnum(b):
            yes_space.add(i)
        
        # Перед многоточием, пробел не нужен
        if i <= n - 3 and s[i:i+3] == "...":
            no_space.add(i)

        # Если длинное тире, то ПРОБЕЛ
        if (a in {"–", "—"}) or (b in {"–", "—"}):
            yes_space.add(i)

        # Если обычный "минус", то классифицируем
        if a == "-" or b == "-":
            kind = classify_minus(s, i)
            if kind == "dash":
                yes_space.add(i)     # тире → пробел
            else:
                no_space.add(i)

    # 2.1. Подготовка к динамическому программированию (разделение на английские/русские буквы /цифры /другое)
    # Группируем подряд идущие символы одного класса в спаны

    spans = []
    cur = []
    prev_category = None

    def category_of(ch):
        if ch in letters_ru:
            return "R"
        if ch in letters_en:
            return "E"
        if ch in digits:
            return "D"
        return "O"

    # На каждом шаге либо продлеваем текущий спан, либо фиксируем и начинаем новый
    for ch in s:
        now = category_of(ch)
        if prev_category is None or now == prev_category:
            cur.append(ch)
            prev_category = now
        else:
            spans.append(("".join(cur), prev_category))
            cur = [ch]
            prev_category = now
    
    # Последний спан тоже фиксируем
    if cur:
        spans.append(("".join(cur), prev_category))

    # 2.2. Динамическое программирование на русских буквах
    # Токенизируем только "R" спаны
    # dp[i] - минимальная стоимость разбиения префикса span[:i]
    # prv[i] - позиция j, откуда пришли в i (для восстановления оптимального разбиения)
    # Стоимость токена определяем по словарю + за каждый разрез (кроме первого токена в спане) - split_penalty

    tokens = []
    for span, category in spans:
        if category != "R":
            tokens.append(span) # Остальные спаны не режем, кладем в один токен
            continue

        len_of_span = len(span)
        dp = [math.inf] * (len_of_span + 1)
        prv = [-1] * (len_of_span + 1)
        dp[0] = 0.0

        for i in range(1, len_of_span+1): # i - правая граница последнего токена
            j0 = max(0, i - max_tok_len) # Не даем токену быть длиннее максимальной длины
            best = math.inf
            best_j = -1
            # Рассматриваем все варианты последнего токена, начиная с длинных
            for j in range(i - 1, j0 - 1, -1):
                tok = span[j:i]
                if j>0:
                    penalty = split_penalty
                else:
                    penalty = 0.0 # Первый токен без штрафа
                cost = dp[j] + token_cost(tok) + penalty  # Минимальная стоимость до j + стоимость токена + штраф 
                if cost < best: # Если меньше лучшего, то обновляем значение
                    best = cost
                    best_j = j
            dp[i] = best # Фиксируем оптимальное значение
            prv[i] = best_j

        # Восстанавливаем оптимальные части по указателям из prv
        parts = []
        i = len_of_span
        while i > 0:
            j = prv[i]
            parts.append(span[j:i])
            i = j
        tokens.extend(reversed(parts))

    # Расставляем пробелы только между словесными (is_word) токенами
    dp_positions = []
    pos = 0
    for i, tok in enumerate(tokens):
        pos += len(tok)
        if i < len(tokens)-1 and is_word(tok) and is_word(tokens[i+1]):
            dp_positions.append(pos)

    # Итоговое расположение пробелов = (Границы между токенами объединенные с yes_space, удаляя no_space позиции) + сортировка
    all_positions = sorted((set(dp_positions) | yes_space) - no_space)   # <<<
    return all_positions

10. Загрузка исходных данных из txt файла

In [26]:
data = []
with open("dataset_1937770_3.txt", 'r') as f:
    for line in f:
        parts = line.strip().split(',', 1)
        if len(parts) == 2:
            try:
                id_val = int(parts[0])
                text_val = parts[1]
                data.append([id_val, text_val])
            except ValueError:
                print(f"Пропускаем строку: {parts[0]}")

task_data = pd.DataFrame(data, columns=['id', 'text'])
task_data

Пропускаем строку: id


,id,text
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку
...,...,...
1000,1000,Янеусну.
1001,1001,Весна-яуженегреюпио.
1002,1002,Весна-скоровырастеттрава.
1003,1003,"Весна-выпосмотрите,каккрасиво."


11. Для каждой строки считаем позиции пробелов и сохраняем как строку

In [27]:
task_data['predicted_positions'] = ''
task_data['predicted_positions'] = task_data['text'].apply(lambda s: str(space_positions(s)))

task_data

,id,text,predicted_positions
0,0,куплюайфон14про,"[5, 7, 10, 12]"
1,1,ищудомвПодмосковье,"[3, 6, 7]"
2,2,сдаюквартирусмебельюитехникой,"[4, 12, 13, 20, 21]"
3,3,новыйдивандоставканедорого,"[5, 10, 18]"
4,4,отдамдаромкошку,"[5, 10]"
...,...,...,...
1000,1000,Янеусну.,[3]
1001,1001,Весна-яуженегреюпио.,"[5, 6, 7, 10, 12, 16, 18]"
1002,1002,Весна-скоровырастеттрава.,"[5, 6, 11, 19]"
1003,1003,"Весна-выпосмотрите,каккрасиво.","[5, 6, 8, 19, 22]"


12. Удаляем столбец и сохраняем в нужном формате

In [28]:
task = task_data.drop(['text'], axis=1)
task

,id,predicted_positions
0,0,"[5, 7, 10, 12]"
1,1,"[3, 6, 7]"
2,2,"[4, 12, 13, 20, 21]"
3,3,"[5, 10, 18]"
4,4,"[5, 10]"
...,...,...
1000,1000,[3]
1001,1001,"[5, 6, 7, 10, 12, 16, 18]"
1002,1002,"[5, 6, 11, 19]"
1003,1003,"[5, 6, 8, 19, 22]"


In [29]:
task_data.to_csv('submissionfull.csv', index=False)

In [30]:
task.to_csv('submission.csv', index=False)